In [110]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)

start_time <- Sys.time()

IN_DIR <- "../../../data/GEO_TCDD_Binary_Dose/input/"
OUT_DIR <- "../../../data/GEO_TCDD_Binary_Dose/output/"

GEO_TCDD_Binary_Dose_DATA_FIL <- "rse_tcdd_data.Rdata"

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")

load(paste(IN_DIR,GEO_TCDD_Binary_Dose_DATA_FIL,sep=""))

In [ ]:
%%R
variances <- apply(assay(final_result), 1, var)
zero_variance_rows <- which(variances == 0)

if (length(zero_variance_rows) > 0) {
    deleted_data <- assay(final_result)[zero_variance_rows, , drop = FALSE]
    save(deleted_data, file=paste0(OUT_DIR, "deleted_data.RData"))
    new_assays <- assay(final_result)[-zero_variance_rows, , drop = FALSE]
    new_row_data <- rowData(final_result)[-zero_variance_rows, , drop = FALSE]

    new_final_result <- SummarizedExperiment(
        assays = SimpleList(counts = new_assays),
        rowData = new_row_data,
        colData = colData(final_result)
    )
    
    final_result <- new_final_result
    
    print(dim(assay(final_result)))
    print(dim(rowData(final_result)))
} else {
    cat("No rows with zero variance found.\n")
}

print(head(assay(final_result)))

[1] 43252   383
[1] 43252    11
                     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
ENSMUSG00000079800.2    0    0    0    0    0    0    0    0   49    49     0
ENSMUSG00000095092.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079192.2    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079794.2    0    0    0    0    0    0    0    0    0     0    50
ENSMUSG00000094799.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000095250.1    0    0    0    0    0   50    0    0    0     0     0
                     [,12] [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20]
ENSMUSG00000079800.2    50     0    50    50     0     0     0     0     0
ENSMUSG00000095092.1     0     0     0     0     0     0     0     0     0
ENSMUSG00000079192.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000079794.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000094799.1     0     0     0     0   

ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0     50     48      0      0      0
                     [,142] [,143] [,144] [,145] [,146] [,147] [,148] [,149]
ENSMUSG00000079800.2      0      0      0      0      0      0      0      0
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,150] [,151] [,152] [,153] [,154] [,155] [,156] [,157]

ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,262] [,263] [,264] [,265] [,266] [,267] [,268] [,269]
ENSMUSG00000079800.2      0      0      0      0      0    100    150     50
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,270] [,271] [,272] [,273] [,274] [,275] [,276] [,277]
ENSMUSG00000079800.2      0      0     50      0     50     40     34     49

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
%%R
keep_samples <- colData(final_result)$dose %in% c(0.00, 30.00)
final_result <- final_result[, keep_samples]
GEO_TCDD_Binary_Dose_data <- colData(final_result)

dose_counts <- table(GEO_TCDD_Binary_Dose_data$dose)

print(dose_counts)

dose_counts_df <- as.data.frame(dose_counts)

colnames(dose_counts_df) <- c("Dose", "Count")

write.csv(dose_counts_df, paste0(OUT_DIR, "dose_counts030.csv"), row.names=FALSE)

DataFrame with 220 rows and 198 columns
    external_id   rail_id       study sra.sample_acc.x sra.experiment_acc
    <character> <integer> <character>      <character>        <character>
1    SRR7817611   2802239   SRP161461       SRS3763870         SRX4669086
2    SRR7817612   2802269   SRP161461       SRS3763870         SRX4669086
3    SRR7817614   2802332   SRP161461       SRS3763872         SRX4669087
4    SRR7817615   2802363   SRP161461       SRS3763868         SRX4669088
5    SRR7817616   2802395   SRP161461       SRS3763868         SRX4669088
...         ...       ...         ...              ...                ...
216  SRR4317656    930487   SRP090688       SRS1721112         SRX2202381
217  SRR4317657    930495   SRP090688       SRS1721113         SRX2202382
218  SRR4317658    930503   SRP090688       SRS1721113         SRX2202382
219  SRR4317659    930510   SRP090688       SRS1721115         SRX2202383
220  SRR4317660    930566   SRP090688       SRS1721115         SRX220238

           <integer>         <character> <integer>           <integer>
1           14559305 2019-05-08 16:15:15 266190129           727965250
2           14821846 2019-05-08 16:15:15 272135592           741092300
3           20958607 2019-05-08 16:15:15 386681562          1047930350
4           14649783 2019-05-08 16:15:15 270011270           732489150
5           14712677 2019-05-08 16:15:15 272283276           735633850
...              ...                 ...       ...                 ...
216         22185618 2017-02-28 15:45:25 654904457          1109280900
217         14055416 2017-02-28 15:45:25 421586591           702770800
218         13639614 2017-02-28 15:45:25 405226312           681980700
219         18673575 2017-02-28 15:45:25 552543054           933678750
220         18752417 2017-02-28 15:45:25 555840036           937620850
    sra.run_total_spots sra.num_reads sra.num_spots          sra.read_info
              <integer>     <integer>     <integer>            <character

                                         <integer>               <numeric>
1                                        565196732                   99.80
2                                        575058122                   99.81
3                                        827957213                   99.64
4                                        588026949                   99.76
5                                        590030308                   99.74
...                                            ...                     ...
216                                      859611848                   98.27
217                                      550633748                   98.26
218                                      534747298                   98.28
219                                      730839638                   98.55
220                                      734131403                   98.56
    recount_qc.bc_auc.unique_% recount_qc.bc_frag.count
                     <numeric>              

                                <integer>                             <integer>
1                                12340743                              18553206
2                                12554723                              18888302
3                                17997081                              26657641
4                                12603672                              18461698
5                                12650973                              18536386
...                                   ...                                   ...
216                              18353927                              28754671
217                              11613299                              18024636
218                              11275394                              17481677
219                              15757135                              24057569
220                              15828817                              24166886
    recount_qc.gene_fc_count_unique.assi

                                                    <integer>
1                                                           0
2                                                           0
3                                                           0
4                                                           0
5                                                           0
...                                                       ...
216                                                         0
217                                                         0
218                                                         0
219                                                         0
220                                                         0
    recount_qc.star.%_of_reads_unmapped:_too_short
                                         <numeric>
1                                             1.28
2                                             1.30
3                                             1.04
4

    recount_qc.star.insertion_average_length2
                                    <integer>
1                                           0
2                                           0
3                                           0
4                                           0
5                                           0
...                                       ...
216                                         0
217                                         0
218                                         0
219                                         0
220                                         0
    recount_qc.star.insertion_rate_per_base
                                  <numeric>
1                                         0
2                                         0
3                                         0
4                                         0
5                                         0
...                                     ...
216                                       0
217   

217                                                        0
218                                                        0
219                                                        0
220                                                        0
    recount_qc.star.number_of_reads_mapped_to_too_many_loci
                                                  <integer>
1                                                    619021
2                                                    633821
3                                                    605238
4                                                    367160
5                                                    370518
...                                                     ...
216                                                  357246
217                                                  234346
218                                                  228740
219                                                  281720
220                                 

219                                                   0
220                                                   0
    recount_qc.star.number_of_splices:_annotated_(sjdb)2
                                               <integer>
1                                                      0
2                                                      0
3                                                      0
4                                                      0
5                                                      0
...                                                  ...
216                                                    0
217                                                    0
218                                                    0
219                                                    0
220                                                    0
    recount_qc.star.number_of_splices:_gc/ag
                                   <integer>
1                                       5807
2           

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
%%R
combined_gender <- ifelse(GEO_TCDD_Binary_Dose_data$Sex != "", GEO_TCDD_Binary_Dose_data$Sex,GEO_TCDD_Binary_Dose_data$gender)

write.table(GEO_TCDD_Binary_Dose_data$project_id,file=paste(OUT_DIR,"GEO_TCDD_Binary_Dose_project_id.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

write.table(combined_gender,file=paste(OUT_DIR,"GEO_TCDD_Binary_Dose_gender.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

write.table(GEO_TCDD_Binary_Dose_data$SRR,file=paste(OUT_DIR,"GEO_TCDD_Binary_Dose_sample_id.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

write.table(GEO_TCDD_Binary_Dose_data$dose,file=paste(OUT_DIR,"GEO_TCDD_Binary_Dose_dose.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

saveRDS(GEO_TCDD_Binary_Dose_data$dose,file=paste(OUT_DIR,"GEO_TCDD_Binary_Dose_detail_vec.Rds",sep=""))

saveRDS(GEO_TCDD_Binary_Dose_data$SRR,file=paste(OUT_DIR,"GEO_TCDD_Binary_Dose_sample_detail_vec.Rds",sep=""))


In [ ]:
%%R
GEO_TCDD_Binary_Dose.df <- final_result %>% SummarizedExperiment::assay() %>% as.data.frame()

ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(GEO_TCDD_Binary_Dose.df))

deleted_ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(deleted_data))

rownames(GEO_TCDD_Binary_Dose.df) <- ensembl_wo_ids

rownames(deleted_data) <- deleted_ensembl_wo_ids

reactome_ensembl_ids <- intersect(ensembl2rxns.df$V1,ensembl_wo_ids)

saveRDS(reactome_ensembl_ids,file=paste(OUT_DIR,"reactome_ensembl_ids.Rds",sep=""))

In [ ]:
%%R
GEO_TCDD_Binary_Dose.df <- GEO_TCDD_Binary_Dose.df[reactome_ensembl_ids,]
saveRDS(GEO_TCDD_Binary_Dose.df,file=paste(OUT_DIR,"GEO_TCDD_Binary_Dose_df.Rds",sep=""))
saveRDS(deleted_data, file=paste(OUT_DIR, "GEO_TCDD_Binary_Dose_format_name_deleted_data.Rds", sep=""))